In [227]:
# -*- coding: utf-8 -*-
import json
import re
import requests
from bs4 import BeautifulSoup

def filter_tags(htmlstr):
    re_div = re.compile('<div(([\s\S])*?)\>', re.I)
    re_divn = re.compile('</div>', re.I)
    re_img = re.compile('<img(([\s\S])*?)\>', re.I)
    re_imgn = re.compile('</img>', re.I)
    re_nn = re.compile('\n\n', re.I)
    s = re_div.sub('', htmlstr)
    s = re_divn.sub('\n', s)
    s = re_img.sub('', s)
    s = re_imgn.sub('\n', s)
    s = re_nn.sub('', s)
    return s

def scp_getxml(url):
    soup = BeautifulSoup(requests.get(url, encoding='utf-8').text, 'lxml')
    head = soup.find('head')
    node = soup.find('div', id="page-content")
    text = "".join('%s' % id for id in (head.select('title') + node.contents))
    return filter_tags(text)

def scp_getjson(record):
    if isinstance(record['Url'], list):
        url = map(lambda x: "http://" + x, record['Url'])
        text = "".join(map(scp_getxml, url))
    else:
        url = "http://" + record['Url']
        text = scp_getxml(url)
    return (json.dumps({
        'Number': record['Number'],
        'Name': record['Name'],
        'HTML': text
    }, ensure_ascii=False))

def scp_export(path, output):
    records = json.load(open(path, "rb"))
    export = "".join(map(scp_getjson, records))
    fileObject = open(output, 'w')
    fileObject.write(export)
    fileObject.close()


In [ ]:
%%time
scp_export(r'main.config.json',r'main.data.json')

In [ ]:
%%time
scp_export(r'branch.config.json',r'branch.data.json')

In [ ]:
%%time
scp_export(r'special.config.json',r'special.data.json')